# LangChain: Models, Prompts and Out put Parsers

## Outline

* Direct API calls to OpenAI
* API calls through LangChain
  * Prompts
  * Models
  * Output parsers

Install Dotenv and OpenAI libraries

In [1]:
# %pip install python-dotenv
# %pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import os

from openai import OpenAI
client = OpenAI()

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())  # read local .env file
# openai.api_key = os.environ['OPENAI_API_KEY']

## Chat API: OpenAI

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0,
    )
    return response.choices[0].message.content

In [4]:
get_completion("What is 1+1")

'1+1 equals 2.'

In [5]:
customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! \
And to make matters worse, \ the warranty don't cover the cost of \ cleaning up me kitchen. \
I need yer help \ right now, matey!
"""

In [6]:
style = """American English\
in a calm and respectful tone
"""

In [7]:
prompt = f"""Translate the text that is delimited by triple backticks into a style that is {style}.\
text: ```{customer_email}```
"""
print(prompt)

Translate the text that is delimited by triple backticks into a style that is American Englishin a calm and respectful tone
.text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, \ the warranty don't cover the cost of \ cleaning up me kitchen. I need yer help \ right now, matey!
```



In [8]:
response = get_completion(prompt)

In [9]:
response

"I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, friend!"

## Chat API: LangChain

In [10]:
# Install langchain library
# %pip install --upgrade langchain

### Model

In [11]:
# langchain.chat_models.ChatOpenAI will be deprecate so use the code in the above cell instead

# Import langchain chat model
# from langchain.chat_models import ChatOpenAI

# chat = ChatOpenAI(temperature=0.0)  # To control the randomness and creativity of the
# # generated text by an LLM, use temperature = 0
# chat

In [12]:
# Install an updated version of langchain-openai
# %pip install -U langchain-openai

In [13]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0.0)  # To control the randomness and creativity of the
# generated text by an LLM, use temperature = 0
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x72dda055e650>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x72dda055d450>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

### Prompt template

In [14]:
template_string = """Translate the text that is delimited by triple backticks \
into a style that is {style}. text: ```{text}```
"""

In [15]:
from langchain.prompts import ChatPromptTemplate
prompt_temaplate = ChatPromptTemplate.from_template(template_string)

In [16]:
prompt_temaplate.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [17]:
prompt_temaplate.messages[0].prompt.input_variables

['style', 'text']

In [18]:
customer_style = """American English in a calm and respectful tone"""

In [19]:
customer_email

"\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, \\ the warranty don't cover the cost of \\ cleaning up me kitchen. I need yer help \\ right now, matey!\n"

In [20]:
customer_messages = prompt_temaplate.format_messages(
    style = customer_style,
    text = customer_email
)

In [21]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [22]:
customer_response = chat(customer_messages)

/home/hassane/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [23]:
print(customer_response.content)

I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, friend!


In [24]:
service_reply = """Hey there customer, the warranty does not cover cleaning expenses for your kitchen \
because it's your fault that you misused your blender by forgetting to put the lid on before starting \
the blender. Tough luck! See ya!
"""

In [25]:
service_style_pirate = """a polite tone that speaks in English Pirate"""

In [26]:
service_messages = prompt_temaplate.format_messages(
    style=service_style_pirate,
    text=service_reply
)
print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [27]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, valued customer! Regrettably, the warranty be not coverin' the cost o' cleanin' yer galley due to yer own negligence. Ye see, 'twas yer own doin' that ye forgot to secure the lid afore startin' the blender. 'Tis a tough break, indeed! Fare thee well, matey!


### Output Parser

In [28]:
# Define how the LLM output should look like
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [29]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. \
    It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [30]:
# from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_temaplate)

input_variables=['style', 'text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'))]


In [31]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [32]:
type(response.content)

str

In [33]:
response.content.get('gift')
# Ann error occurs because response.content is a just a very long string not a dictionnary

AttributeError: 'str' object has no attribute 'get'

### Parse the LLm output string into a Python dictionnary

In [34]:
from langchain.output_parsers import (
    ResponseSchema, StructuredOutputParser
)

In [35]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown."
)
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? If this  information is not found, output -1."
)
price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price, and output them as a comma separated Python list."
)

response_schemas = [
    gift_schema,
    delivery_days_schema,
    price_value_schema
]

In [36]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [37]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this  information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [38]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(
    text=customer_review,
    format_instructions=format_instructions
)

In [39]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn.     It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" a

In [40]:
response = chat(messages)

In [41]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [42]:
output_dict = output_parser.parse(response.content)

In [43]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [44]:
type(output_dict)

dict

In [45]:
print(output_dict.get('gift'))
print(output_dict.get('delivery_days'))
print(output_dict.get('price_value'))

True
2
It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
